### Task 2 - Data Wrangling

##### I will use the wikipedia data as my main dataset, and the spaceX-API data will be used to fill in the missing values of the wikipedia dataset.

In [1]:
import pandas as pd

In [2]:
# Load data collected from task 1
api_df = pd.read_csv("datasets/api_dataset.csv")
wp_df = pd.read_csv("datasets/wp_dataset.csv")

In [3]:
# Remove the entries that indicate no attempt to land the booster
wp_df = wp_df[wp_df["Outcome"] != "No attempt"]

In [4]:
# Check the number of entries in the dataset
len(wp_df)

232

In [5]:
# Check the percentage of missing values for each column
missing =  wp_df.isna().sum()*100/len(wp_df)
missing = pd.DataFrame(missing.reset_index())
missing = missing.rename(columns={"index": "ColumnName", 0:"MissingValuePercent"})
missing

,ColumnName,MissingValuePercent
0,FlightNumber,0.000000
1,Date,0.000000
2,Hour,0.000000
3,Minute,0.000000
4,BoosterVersion,0.000000
5,PayloadMass,6.465517
6,Orbit,0.000000
7,LaunchSite,0.000000
8,Serial,0.862069
9,Block,0.862069


In [6]:
# Fill the Serial and Block missing values by matching the Date column with the SpaceX API datasets
merge_df = wp_df.merge(api_df[["Date", "Serial", "Block"]], on="Date", how="left")

wp_df["Serial"].fillna(merge_df["Serial_y"], inplace=True)
wp_df["Block"].fillna(merge_df["Block_y"], inplace=True)
wp_df["Serial"].fillna(merge_df["Serial_y"], inplace=True)

In [7]:
# Add year column
wp_df["Year"] = wp_df.Date.str.split("-").str[0]

# Add month column
wp_df["Month"] = wp_df.Date.str.split("-").str[1]

# Add day column
wp_df["Day"] = wp_df.Date.str.split("-").str[2]

# Add Flight column - the number of flights with the core
wp_df["Flights"] = wp_df.groupby("Serial").cumcount()+1

# Add Reused column - whether the core is reused
wp_df["Reused"] =  wp_df["Flights"] > 1

In [8]:
# Fill the PayloadMass missing values by the mean value of payload mass for the same (Orbit, Block, Outcome) pairs

# compute average mass value of each (Orbit, Block, Outcome) pair
mean_mass = round(wp_df.groupby(["Orbit", "Block", "Outcome"])["PayloadMass"].mean().reset_index())
mean_mass

merge_df = wp_df.merge(mean_mass[["Orbit", "Block", "PayloadMass", "Outcome"]], on=["Orbit", "Block","Outcome"], how="left")
merge_df = merge_df.reset_index()
wp_df = wp_df.reset_index()

# fill the missing values
wp_df["PayloadMass"].fillna(merge_df["PayloadMass_y"], inplace=True)
wp_df = wp_df.drop("index", axis=1)


In [9]:
# Recheck the number of missing values
missing = wp_df.isna().sum()
missing

FlightNumber      0
Date              0
Hour              0
Minute            0
BoosterVersion    0
PayloadMass       0
Orbit             0
LaunchSite        0
Serial            0
Block             0
Longitude         0
Latitude          0
Outcome           0
LandingPlace      0
Year              0
Month             0
Day               0
Flights           0
Reused            0
dtype: int64

In [10]:
# Check the type of each column
wp_df.dtypes

FlightNumber        int64
Date               object
Hour                int64
Minute              int64
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Serial             object
Block             float64
Longitude         float64
Latitude          float64
Outcome            object
LandingPlace       object
Year               object
Month              object
Day                object
Flights             int64
Reused               bool
dtype: object

In [11]:
# Revise the columns' type
string_to_boolean = {"True":1, "False":0}
wp_df['Outcome'] = wp_df['Outcome'].map(string_to_boolean)
wp_df['Year'] = wp_df['Year'].astype(int)
wp_df['Month'] = wp_df['Month'].astype(int)
wp_df['Day'] = wp_df['Day'].astype(int)

In [12]:
# wp_df.to_csv("datasets/falcon9_dataset.csv", sep=",", index=False)